In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env.localhost")

True

# S3 init

In [3]:
from functools import lru_cache
import os

import boto3

API_SITE_BUCKET = os.getenv("API_SITE_ID")


@lru_cache
def get_s3_client():
    _session = boto3.session.Session()
    return _session.client(
        "s3",
        region_name=os.getenv("DO_REGION_NAME"),
        endpoint_url=os.getenv("DO_ENDPOINT_URL"),
        aws_access_key_id=os.getenv("DO_SPACES_KEY"),
        aws_secret_access_key=os.getenv("DO_SPACES_SECRET"),
    )


def presigned_url_to_put(
    key, client=None, mime_type=None, bucket=API_SITE_BUCKET, expires_in=300
):
    return client.generate_presigned_url(
        ClientMethod="put_object",
        Params={"Bucket": bucket, "Key": key, "ContentType": mime_type},
        ExpiresIn=expires_in,
    )

s3client = get_s3_client()

response = s3client.list_buckets()
for space in response['Buckets']:
    print(space['Name'])
print(f"\nusing Bucket {API_SITE_BUCKET}\n")
response = s3client.list_objects(Bucket=API_SITE_BUCKET)
for obj in response['Contents']:
    print(obj['Key'])

nmdc-runtime
polyneme

using Bucket nmdc-runtime

do/03c3-5kqv-57
do/08j73-wn755
do/0cgc-qt5f-48
do/1bdd-ncsd-02
do/22e0-wqt8-69
do/2wem-dzg1-84
do/4kr3-xa85-27
do/55ss-n50r-11
do/5acs-zdnm-73
do/5f0b-pbs6-30
do/720v-9dag-24
do/85k8-agkq-63
do/8a8w-s0qw-11
do/8e7t-nd51-05
do/acvg-nm6k-61
do/aea1-qvxa-25
do/ajrh-6sff-63
do/avgy-htny-59
do/azs9-t5dd-49
do/c4n1-9a3d-06
do/c6hs-4rzh-31
do/dnk5-yqqp-09
do/e27s-9fsv-88
do/ecvr-mb80-57
do/ewfm-fh80-44
do/f7bq-ngn0-55
do/ges8-mq5d-58
do/gygb-zv5w-13
do/j9cm-9hx6-69
do/kyew-jfne-27
do/pjwn-8yhy-54
do/qzna-7hxj-86
do/s50g-0rq0-58
do/tac5-5yh9-04
do/veh6-4naq-38
do/vfav-x4w5-09
do/y2s2-p1ag-52
do/yqm2-ehg2-11
gold_etl/nmdc_database.json.zip
test/test.txt


# Site Client Session

In [4]:
from nmdc_runtime.util import nmdc_jsonschema, nmdc_jsonschema_validate

In [5]:
from nmdc_runtime.pipelines.core import run_config_frozen__preset_normal_env
from nmdc_runtime.resources.mongo import get_mongo


mongo = get_mongo(run_config_frozen__preset_normal_env)
db = mongo.db

In [6]:
from nmdc_runtime.pipelines.core import run_config_frozen__preset_normal_env
from nmdc_runtime.resources.core import get_runtime_api_site_client


client = get_runtime_api_site_client(run_config_frozen__preset_normal_env)

In [7]:
from pathlib import Path

from dagster import build_solid_context
from nmdc_runtime.solids.core import local_file_to_api_object as lftao

context = build_solid_context(
    resources={"mongo": mongo, "runtime_api_site_client": client}
)

storage_path = "../src/data/2021-07-02-study-changes.csv"


def local_file_to_api_object(file_info):
    return lftao(context, file_info)

#obj = local_file_to_api_object({"storage_path": storage_path, "mime_type": 'text/csv'})

#doc = db.objects.find_one({"id": obj["id"]})
#assert doc["name"] == Path(storage_path).name

In [ ]:
!echo '{"hello": "donny"}' > test.json

In [ ]:
rv = client.put_object_in_site({"mime_type": "application/json", "name": "test.json"})

In [ ]:
op = rv.json()

In [ ]:
import put_object

rv = put_object("test.json", op["metadata"]["url"])
rv

In [ ]:
from nmdc_runtime.util import drs_object_in_for

result = drs_object_in_for("test.json", op)

In [ ]:
import json
from pprint import pprint

op_patch = {"done": True, "result": result}

In [ ]:
rv = client.update_operation(op["id"], op_patch)

In [ ]:
pprint(rv.json())

# GSP schema  / Cordra stuff

Need "id" in payload, e.g.:
```
"results": [
    {
      "id": "test/activity",
      "type": "Schema",
      "content": {
        "name": "Activity",
        "schema": collschemas["activity_set"]
      }
    }
  ]
```

In [ ]:
from time import time
import os

tic = time()

from dotenv import load_dotenv
load_dotenv(os.path.expanduser("~/.nmdc_mongo.env"))

In [ ]:
os.environ["NMDC_JSON_SCHEMA_FILE"] = "/Users/dwinston/Desktop/nmdc.schema.gsp.json"

In [ ]:
import json
import re
from toolz import assoc_in, dissoc
from zipfile import ZipFile

from mongospawn.schema import collschemas_for

from nmdc_mongo import (
    add_to_db,
    correct_metaP_doc,
    dbschema,
    fetch_and_validate_json,
    fetch_conform_and_persist_from_manifest,
    fetch_json,
    get_db,
    reset_database,
    snake_case_set_name
)

In [ ]:
###########################
# Adjustments for GSP below
###########################

defined_object_names = set(dbschema["definitions"])

set_for_object_name = {
    spec["items"]["$ref"].split("#/definitions/")[-1]: set_name
    for set_name, spec in dbschema["properties"].items()
}

existing_set_names = set(dbschema["properties"])

for object_without_set in (defined_object_names - set(set_for_object_name.keys())):
    proposed_set_name = snake_case_set_name(object_without_set)
    if proposed_set_name not in existing_set_names:
        dbschema["properties"][proposed_set_name] = {
            "description": (f"This property links a database object to the set of"
                            f" {object_without_set} objects within it."),
            "items": {"$ref": f"#/definitions/{object_without_set}"},
            "type": "array",
        }
        
dbschema = assoc_in(dbschema, ["definitions", "ControlledTermValue", "properties", "term", "type"], "string")
del dbschema["definitions"]["ControlledTermValue"]["properties"]["term"]["$ref"]

# 'k' not capitalized upstream perhaps. should conform!
#dbschema = assoc_in(dbschema, ["definitions", "MetagenomeAssembly", "properties", "scaf_l_gt50k", "type"], "number")

In [ ]:
collschemas = collschemas_for(dbschema)

# Reconstruct
set_for_object_name = {
    spec["items"]["$ref"].split("#/definitions/")[-1]: set_name
    for set_name, spec in dbschema["properties"].items()
}

In [ ]:
sorted(collschemas.keys())

In [ ]:
collschemas["biosample_set"]

In [ ]:
import requests

In [ ]:
rv = requests.post("http://localhost:8080/auth/token",
              {"grant_type": "password", "username": "admin", "password": "nmdcrulez"})

In [ ]:
rv.json()

In [ ]:
auth_header = {"Authorization": f'Bearer {rv.json()["access_token"]}'}

In [ ]:
auth_header

In [ ]:
rv = requests.post("http://localhost:8080/uploadObjects", json={
    "results": [
        {
          "type": "Schema",
          "content": {
            "name": "Biosample",
            "schema": collschemas["biosample_set"]
          }
        }
      ]
}, headers=auth_header)

In [ ]:
rv

In [ ]:
import json
with open('/Users/dwinston/Desktop/cordra-upload.json','w') as f:
    json.dump({
        "results": [
            {
              "id": "test/activity",
              "type": "Schema",
              "content": {
                "name": "Activity",
                "schema": collschemas["activity_set"]
              }
            }
          ]
    }, f, indent=2)

In [ ]:
import json
with open('/Users/dwinston/Desktop/cordra-upload.json','w') as f:
    json.dump({
        "results": [
            {
              "id": "test/study",
              "type": "Schema",
              "content": {
                "name": "Study",
                "schema": collschemas["study_set"]
              }
            }
          ]
    }, f, indent=2)

In [ ]:
rv = requests.get("http://localhost:8080/search?query=type:%22Schema%22", headers=auth_header)

In [ ]:
from pprint import pprint

template = rv.json()
del template["pageNum"]
del template["pageSize"]
del template["size"]
del template["results"][0]
del template["results"][0]
template["results"][0]["id"] = "test/abcd1234"
template["results"][0]["content"]["name"] = "Document2"
template["results"][0]["content"]["schema"]["title"] = "Document2"
del template["results"][0]["content"]["identifier"]

pprint(template)

In [ ]:
import json
with open('/Users/dwinston/Desktop/cordra-upload.json','w') as f:
    json.dump(template, f, indent=2)

In [ ]:
!cat /Users/dwinston/Desktop/cordra-upload.json

In [ ]:
rv = requests.post("http://localhost:8080/uploadObjects", json=template, headers=auth_header)
rv